In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import time
import pandas as pd
import torch
from sklearn.metrics import accuracy_score

dataset = load_dataset("NicolaiSivesind/human-vs-machine", "research_abstracts_labeled")['test'][0:50]
x_test, y_test = dataset['text'], dataset['label']
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector")
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector")

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset human-vs-machine (/home/codespace/.cache/huggingface/datasets/NicolaiSivesind___human-vs-machine/research_abstracts_labeled/0.0.0/9e9ff0b78fa974ae55166fbed3b9032d432c39f2e76909d68bdd53cb4ea313de)
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.09s/it]


### 2. Evaluate Performance

In [5]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

base_model_size = (param_size + buffer_size) / 1024**2
print('Model size: {:.3f}MB'.format(base_model_size))

Model size: 11453.867MB


In [6]:
y_pred = []
times = []
sigmoid = torch.nn.Sigmoid()
for idx, text in enumerate(x_test):
    input = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    st = time.time()
    output = model(**input).logits
    predicted_class = torch.argmax(output).item()
    y_pred.append(predicted_class)
    times.append(time.time() - st)

print(y_pred)
print(times)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[0, 1, 0, 1, 1, 1, 0, 0, 1, 1]
[3.0076465606689453, 2.563746690750122, 7.519760847091675, 4.173773765563965, 9.64520001411438, 6.830012559890747, 4.011502742767334, 3.0249407291412354, 1.7653095722198486, 1.77117919921875]


In [3]:
y_pred = [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1]
accuracy_score(y_test, y_pred)

0.64

In [5]:
times = pd.Series([3.899423122406006, 3.0144855976104736, 8.07302212715149, 6.465184926986694, 3.150974988937378, 2.902872085571289, 6.113534450531006, 5.451525926589966, 8.014659643173218, 5.68673038482666, 
                  1.4908013343811035, 1.391730785369873, 7.938936948776245, 7.7671730518341064, 7.570752382278442, 6.162149667739868, 7.994791507720947, 6.498159170150757, 7.302468538284302, 5.080890655517578, 
                4.976664781570435, 2.71449613571167, 2.98722767829895, 2.6232614517211914, 11.119134187698364, 5.91392183303833, 6.074051141738892, 5.266791105270386, 7.89971923828125, 6.698412895202637, 
                 12.17499589920044, 8.233731031417847, 6.3944008350372314, 4.704039573669434, 3.8241074085235596, 3.3292598724365234, 1.5289897918701172, 1.7963066101074219, 3.070106029510498, 2.5719656944274902, 
                  3.0076465606689453, 2.563746690750122, 7.519760847091675, 4.173773765563965, 9.64520001411438, 6.830012559890747, 4.011502742767334, 3.0249407291412354, 1.7653095722198486, 1.77117919921875])
                 
times.describe()


count    50.000000
mean      5.203698
std       2.606995
min       1.391731
25%       3.009356
50%       5.173841
75%       7.184355
max      12.174996
dtype: float64